# Efficient Variants
Variant 1: Random Subset of Features 

Variant 2: Relaxing the Upper and Lower Bound Difference


In [7]:
import time
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict, deque

In [ ]:
#!/usr/bin/env python3
"""
gosdt_scratch.py

Pure‑Python (NumPy‑only) re‑implementation of the GOSDT algorithm
with two optional efficiency tweaks:

    * random feature subset per split         (use_feature_subsample = True)
    * relaxed hierarchical/pruning bound      (relax_bound           = True)

Written from the descriptions in:
  – Lin et al.,  ICML’20  paper
  – Rudin course notes (Modern Decision‑Tree Optimisation)

"""



# Utility: binarise data (exact mid‑point‑split encoding)
def binarise(X_orig, thresholds=None):
    """
    Turn a real‑valued matrix (n × p) into binary indicators x ≤ θ.

    Returns X_bin (n × m) and a list ‘meta’ with (orig_col, θ) pairs.
    If ‘thresholds’ is passed, we re‑use those (enables test split).
    """
    n, p = X_orig.shape
    if thresholds is None:                         # build once from training data
        thresholds = []
        for j in range(p):
            uniq = np.unique(X_orig[:, j])
            if len(uniq) == 1:                     # constant → skip
                continue
            mids = (uniq[:-1] + uniq[1:]) / 2.0
            thresholds.extend((j, float(t)) for t in mids)

    # build binary design
    X_bin = np.empty((n, len(thresholds)), dtype=np.uint8)
    for k, (j, t) in enumerate(thresholds):
        X_bin[:, k] = (X_orig[:, j] <= t).astype(np.uint8)

    return X_bin, thresholds

#  Node object for the search tree
class Node:
    __slots__ = (
        "bitmask", "depth", "pred", "lb", "ub",
        "left", "right", "split_feature",
    )

    def __init__(self, bitmask, depth):
        self.bitmask = bitmask            # frozenset of row indices
        self.depth   = depth
        self.pred    = None               # majority label (set when leaf)
        self.lb = 0.0                     # lower bound on objective
        self.ub = math.inf                # upper bound on objective
        self.left = self.right = None
        self.split_feature = None

    # leaf test
    def is_leaf(self):
        return self.left is None and self.right is None



class GOSDTScratch:
    def __init__(
        self,
        lam=0.02,
        depth_limit=5,
        use_feature_subsample=False,
        feat_rate=0.5,
        relax_bound=False,
        eps=0.01,
        random_state=0,
    ):
        self.lam = lam
        self.D   = depth_limit
        self.use_feature_subsample = use_feature_subsample
        self.feat_rate = feat_rate
        self.relax_bound = relax_bound
        self.eps = eps
        self.rng = random.Random(random_state)

        # attributes filled after fit()
        self.root       = None
        self.n_features = None
        self.y          = None


    # public API
    def fit(self, X_bin, y):
        """X_bin must be binary 0/1 matrix (already binarised)."""
        n, m = X_bin.shape
        self.n_features = m
        self.y = y

        # pre‑compute indices of positives / negatives per column
        self.col_pos = [set(np.nonzero(X_bin[:, j])[0]) for j in range(m)]
        self.col_neg = [set(range(n)) - col for col in self.col_pos]

        # global best
        self.best_obj = math.inf
        self.best_tree = None

        # memoisation: map bitmask → (lb, ub)  (dynamic programming reuse)
        self.memo = {}

        # priority queue (deque works: process shallow nodes first)
        master_set = frozenset(range(n))
        self.root = Node(master_set, depth=0)
        self._init_bounds(self.root)

        Q = deque([self.root])
        while Q:
            node = Q.popleft()

            # bound + prune
            if node.lb >= self.best_obj:
                continue

            # solve sub‑problem (branch or leaf)
            solved = self._process_node(node)
            if solved:
                if node.ub < self.best_obj:
                    self.best_obj = node.ub
                    self.best_tree = node
            else:
                # push viable children in BFS order
                if node.left.lb < self.best_obj:
                    Q.append(node.left)
                if node.right.lb < self.best_obj:
                    Q.append(node.right)

        # store final root (best_tree is a pointer inside that structure)
        self.root = self.best_tree

    def predict(self, X_bin):
        out = np.empty(X_bin.shape[0], dtype=int)
        for i in range(len(out)):
            out[i] = self._predict_row(self.root, X_bin[i])
        return out


    # helpers

    def _predict_row(self, node, x_row):
        while not node.is_leaf():
            node = node.left if x_row[node.split_feature] else node.right
        return node.pred

    # --------------------  bounds initialisation  --------------------
    def _init_bounds(self, node):
        """Compute trivial leaf objective for upper bound; lower bound = 0."""
        idx = list(node.bitmask)
        if not idx:                       # empty set (shouldn't happen)
            node.lb = node.ub = 0.0
            return

        y_sub = self.y[idx]
        pos = np.count_nonzero(y_sub)
        neg = len(idx) - pos
        minority = min(pos, neg)

        node.pred = 1 if pos >= neg else 0
        loss_leaf = minority / len(self.y)            # global normalisation
        node.lb = 0.0                                # best possible loss after splits
        node.ub = loss_leaf + self.lam               # make‑it‑leaf objective

        # relaxed bound tweak: enlarge lb to prune more aggressively
        if self.relax_bound:
            node.lb += self.eps

        self.memo[node.bitmask] = (node.lb, node.ub)

    # --------------------  branch‑and‑bound step  --------------------
    def _process_node(self, node):
        """
        Returns True if node is final (leaf),
        False if it was split and children inserted.
        """
        # trivial cases
        if node.depth >= self.D:
            node.lb = node.ub                         # cannot split further
            return True
        if node.ub - node.lb <= 1e-12:                # perfect or pruned
            return True

        best_feature = None
        best_l, best_r = None, None
        best_obj = math.inf

        # candidate feature set ---------------------------------------
        feat_indices = range(self.n_features)
        if self.use_feature_subsample:
            k = max(1, int(self.feat_rate * self.n_features))
            feat_indices = self.rng.sample(list(feat_indices), k)

        for j in feat_indices:
            left_set  = node.bitmask & self.col_pos[j]
            right_set = node.bitmask & self.col_neg[j]
            if not left_set or not right_set:         # useless split
                continue

            # reuse children bounds if already solved
            l_lb, l_ub = self.memo.get(left_set, (None, None))
            r_lb, r_ub = self.memo.get(right_set, (None, None))

            # child not seen → create temp Node for bounds
            if l_lb is None:
                l_temp = Node(left_set, node.depth + 1)
                self._init_bounds(l_temp)
                l_lb, l_ub = l_temp.lb, l_temp.ub
            if r_lb is None:
                r_temp = Node(right_set, node.depth + 1)
                self._init_bounds(r_temp)
                r_lb, r_ub = r_temp.lb, r_temp.ub

            lb_split = l_lb + r_lb
            ub_split = l_ub + r_ub

            if ub_split < best_obj:
                best_obj = ub_split
                best_feature = j
                best_l, best_r = (left_set, (l_lb, l_ub)), (right_set, (r_lb, r_ub))

        # no admissible split improves upper bound ⇒ keep as leaf
        if best_feature is None or best_obj >= node.ub - 1e-12:
            node.lb = node.ub
            return True

        # materialise children nodes
        l_set, (l_lb, l_ub) = best_l
        r_set, (r_lb, r_ub) = best_r
        node.split_feature = best_feature

        node.left  = Node(l_set, node.depth + 1)
        node.right = Node(r_set, node.depth + 1)
        node.left.lb,  node.left.ub  = l_lb, l_ub
        node.right.lb, node.right.ub = r_lb, r_ub

        # update current bounds
        node.lb = node.left.lb + node.right.lb + self.lam   # +λ for this split
        node.ub = node.left.ub + node.right.ub + self.lam

        # write in memo
        self.memo[node.bitmask] = (node.lb, node.ub)
        return False


#  Demo on 50‑sample Wine subset, four scenarios

def run_demo():
    # raw wine data (from UCI): embed CSV to stay std‑lib, or generate quickly
    import urllib.request, io, csv
    URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
    with urllib.request.urlopen(URL) as resp:
        data = np.loadtxt(io.BytesIO(resp.read()), delimiter=",")
    y_raw  = data[:, 0].astype(int) - 1                # classes 0,1,2
    X_raw  = data[:, 1:]

    rng = np.random.default_rng(0)
    idx = rng.choice(len(X_raw), 50, replace=False)
    X_raw, y_raw = X_raw[idx], y_raw[idx]

    # 70 / 30 split
    n = len(X_raw)
    perm = rng.permutation(n)
    train_idx = perm[: int(0.7 * n)]
    test_idx  = perm[int(0.7 * n):]

    X_train_raw, y_train = X_raw[train_idx], y_raw[train_idx]
    X_test_raw,  y_test  = X_raw[test_idx],  y_raw[test_idx]

    # binarise with thresholds learned on training part
    X_train_bin, thresholds = binarise(X_train_raw)
    X_test_bin, _           = binarise(X_test_raw, thresholds)

    configs = [
        dict(name="Baseline",          subsample=False, relax=False),
        dict(name="Random subset",     subsample=True,  relax=False),
        dict(name="Relaxed bound",     subsample=False, relax=True),
        dict(name="Both",              subsample=True,  relax=True),
    ]

    accs, times = [], []
    for cfg in configs:
        t0 = time.perf_counter()
        clf = GOSDTScratch(
            lam=0.02,
            depth_limit=5,
            use_feature_subsample=cfg["subsample"],
            feat_rate=0.5,
            relax_bound=cfg["relax"],
            eps=0.01,
            random_state=0,
        )
        clf.fit(X_train_bin, y_train)
        preds = clf.predict(X_test_bin)
        runtime = time.perf_counter() - t0
        acc = (preds == y_test).mean()
        accs.append(acc)
        times.append(runtime)
        print(f"{cfg['name']:>14s}  acc={acc:.2f}  time={runtime*1000:.1f} ms")

    # chart
    xs = np.arange(len(configs))
    width = 0.35
    fig, ax1 = plt.subplots(figsize=(8,4))
    ax2 = ax1.twinx()
    ax1.bar(xs - width/2, accs, width, label="Accuracy")
    ax2.bar(xs + width/2, times, width, color="orange", label="Runtime (s)")
    ax1.set_xticks(xs)
    ax1.set_xticklabels([c["name"] for c in configs])
    ax1.set_ylabel("Accuracy")
    ax2.set_ylabel("Seconds")
    ax1.set_title("Scratch‑built GOSDT variants on 50‑sample wine subset (λ = 0.02)")
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    fig.tight_layout()
    plt.show()


# run when executed directly ---------------------------------------------------
if __name__ == "__main__":
    run_demo()


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'